# A Pan Cancer Atlas
## A Study In 12 Chapters, A Prologue and An Epilogue

### Methodology
Using the high throughput data and employing transcriptomics analyses on the targeted dataset, we can evaluate the differentially expressed genes (DEGs), which can affect many biological pathways through their dysregulation. However, these studies cannot describe the quantitative association between genes and failed to give us a deep understanding of complex hierarchical networks and how the genes in critical pathways interact. Alternatively, we can utilize better methods to understand that quantitative association targets, not a set of candidate genes but a network of genes associated with biological processes. For instance, we can employ protein-protein or gene-gene interaction (PPI and GGI), annotated pathways (e.g., KEGG and PANTHER), and gene ontology terms. In this study, we used the RNA co-expression network analysis to find our datasets' most critical gene cluster. Weighted Gene Co-Expression Network Analysis (WGCNA). We aim to evaluate the DEGs and PPI analysis on TCGA datasets in twelve cancer types and study those candidate genes in the associated pathways. Therefore, by using WGCNA, we can score those findings based on the merged and unmerged gene clusters that WGCNA evaluated. Moreover, we can use a machine learning algorithm trained based on the adjacency matrix, which WGCNA constructed from, and associate that module with clinical data. This module will aptly explain at the end of this proposal. 
First and foremost, we should evaluate our DEGs. In this proposal, we use the LUAD-TCGA project, but the table of the candidate projects of twelve cancer types will be provided in the following.

In [36]:
# Essentials

#install.packages("dplyr")
#install.packages("ggplot2")
#install.packages("pheatmap")
#install.packages("parallel")

library(dplyr)
library(ggplot2)
library(pheatmap)
library(parallel)

with our custom code we downloaded LUAD data set from TCGA portal and make a tidy data sets based on each sample file


In [19]:
#options(stringsAsFactors = F)
setwd("~/desktop/THCA/")
ex <- read.delim("~/Desktop/THCA/ex_LUAD.tsv")
cn <- ex

In [20]:
dim(cn)
cn <- distinct(cn, gene_name, .keep_all = TRUE)
dim(cn)
# lose 1233 genes

[1] 60660   555

[1] 59427   555

In [22]:
rownames(cn) <- cn$gene_name
cn <- cn[,-1:-2]
head (cn)

,TCGA.55.6972.11A,TCGA.86.8075.01A,TCGA.69.7763.01A,TCGA.38.4631.01A,TCGA.05.5425.01A,TCGA.MP.A4T7.01A,TCGA.J2.A4AD.01A,TCGA.38.4625.11A,TCGA.55.8299.01A,TCGA.05.4395.01A,⋯,TCGA.69.7978.01A,TCGA.05.4398.01A,TCGA.50.5931.11A,TCGA.49.AARE.01A,TCGA.55.6971.01A,TCGA.NJ.A4YF.01A,TCGA.55.8616.01A,TCGA.53.7624.01A,TCGA.95.7948.01A,TCGA.44.4112.01B
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,573,11426,1624,2567,2393,1456,3719,1990,2143,3131,⋯,3153,6052,577,5209,1369,1267,2718,2020,3087,591
2,1,3,0,1,0,0,2,9,0,0,⋯,3,3,1,2,0,0,0,1,0,20
3,619,2428,1588,1771,2238,1851,2154,1984,1452,3193,⋯,2063,3726,941,1229,703,3955,1165,1419,2389,383
4,370,1236,498,354,640,650,1319,608,737,1326,⋯,683,1336,275,666,479,362,639,891,1171,871
5,68,888,114,488,468,378,563,161,430,483,⋯,375,1243,36,358,140,210,293,813,282,462
6,2599,1375,743,343,1232,1686,357,12548,1304,580,⋯,2700,3469,2662,909,1004,492,525,322,240,204


Making a data frame with sample ID and cancer type as `samples` and replacing for `cn` dataframe


In [23]:
sampleID <- read.delim("~/desktop/THCA/TCGA-LUAD/Data/gdc_sample_sheet.2022-06-11.tsv")
i1 <- sampleID$Data.Type == "Gene Expression Quantification"
samples <- sampleID[i1, , drop = F]
samples <- samples[,7:8]
a <- colnames(cn)
head (cn)

,TCGA.55.6972.11A,TCGA.86.8075.01A,TCGA.69.7763.01A,TCGA.38.4631.01A,TCGA.05.5425.01A,TCGA.MP.A4T7.01A,TCGA.J2.A4AD.01A,TCGA.38.4625.11A,TCGA.55.8299.01A,TCGA.05.4395.01A,⋯,TCGA.69.7978.01A,TCGA.05.4398.01A,TCGA.50.5931.11A,TCGA.49.AARE.01A,TCGA.55.6971.01A,TCGA.NJ.A4YF.01A,TCGA.55.8616.01A,TCGA.53.7624.01A,TCGA.95.7948.01A,TCGA.44.4112.01B
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,573,11426,1624,2567,2393,1456,3719,1990,2143,3131,⋯,3153,6052,577,5209,1369,1267,2718,2020,3087,591
2,1,3,0,1,0,0,2,9,0,0,⋯,3,3,1,2,0,0,0,1,0,20
3,619,2428,1588,1771,2238,1851,2154,1984,1452,3193,⋯,2063,3726,941,1229,703,3955,1165,1419,2389,383
4,370,1236,498,354,640,650,1319,608,737,1326,⋯,683,1336,275,666,479,362,639,891,1171,871
5,68,888,114,488,468,378,563,161,430,483,⋯,375,1243,36,358,140,210,293,813,282,462
6,2599,1375,743,343,1232,1686,357,12548,1304,580,⋯,2700,3469,2662,909,1004,492,525,322,240,204


Converting "." to "-" in sample IDs

In [24]:
a <- sub('\\.', '-', a)
a <- sub('\\.', '-', a)
a <- sub('\\.', '-', a)
a <- sub('\\.', '-', a)

Adding sample type into the `colnames`

In [25]:
b <- data.frame(Sample.ID = a)
c <- merge(b, samples, by.x = "Sample.ID", by.y = "Sample.ID", sort = F)
colnames (cn) <- c$Sample.Type
head(cn)

,Solid Tissue Normal,Primary Tumor,Primary Tumor.1,Primary Tumor.2,Primary Tumor.3,Primary Tumor.4,Primary Tumor.5,Solid Tissue Normal.1,Primary Tumor.6,Primary Tumor.7,⋯,Primary Tumor,Primary Tumor.1,Solid Tissue Normal,Primary Tumor.2,Primary Tumor.3,Primary Tumor.4,Primary Tumor.5,Primary Tumor.6,Primary Tumor.7,Primary Tumor.8
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,573,11426,1624,2567,2393,1456,3719,1990,2143,3131,⋯,3153,6052,577,5209,1369,1267,2718,2020,3087,591
2,1,3,0,1,0,0,2,9,0,0,⋯,3,3,1,2,0,0,0,1,0,20
3,619,2428,1588,1771,2238,1851,2154,1984,1452,3193,⋯,2063,3726,941,1229,703,3955,1165,1419,2389,383
4,370,1236,498,354,640,650,1319,608,737,1326,⋯,683,1336,275,666,479,362,639,891,1171,871
5,68,888,114,488,468,378,563,161,430,483,⋯,375,1243,36,358,140,210,293,813,282,462
6,2599,1375,743,343,1232,1686,357,12548,1304,580,⋯,2700,3469,2662,909,1004,492,525,322,240,204


## Differentially Expressed Genes (DEGs)

#### Data Prepration is ready
#### Next step will be performed by DESeq2 library

In [ ]:
install.package("DESeq2")


In [ ]:
dim(cn)
#install.packages("DESeq2")
library(DESeq2)

gr <- factor(c$Sample.Type)
colSums(cn) #Through this we can demonstrates 
colData <- data.frame(group = gr, type = "paired-end")
cds <- DESeqDataSetFromMatrix(cn, colData, design = ~group)
cds <- DESeq(cds)
cnt <- log2(1+(counts(cds, normalize = T))) #getting normalized counts

In [ ]:
#DEGs
dif <- results(cds, c("group", "Solid Tissue Normal" , "Primary Tumor"))
write.table(dif, "~/desktop/dif.csv",  quote = F, col.names = T, row.names = T, sep = "\t")

#checkpoint
sorted_dif <- data.frame(results(cds, c("group", "Solid Tissue Normal" , "Primary Tumor")))
sorted_dif$padj <- p.adjust(sorted_dif$pvalue, method = "BH")
sorted_dif <- sorted_dif[order(sorted_dif$padj),]

UP  <- subset(sorted_dif, log2FoldChange > 1  & padj < 0.05)
DOWN  <- subset(sorted_dif, log2FoldChange < -1  & padj < 0.05)

dim(UP); dim(DOWN)

The up and down regulated genes exported into two separate files.

In [ ]:
write.table(UP, "~/desktop/Up-regulated.csv",  quote = F, col.names = T, row.names = T, sep = "\t")
write.table(DOWN, "~/desktop/Down-regulated.csv",  quote = F, col.names = T, row.names = T, sep = "\t")
head(UP)
head (DOWN)

For Checking the quality of the data we utilize several statistical languages such as boxplot analysis, principal component analysis, volcano plot, and Correlation heatmap analysis.

In [ ]:
boxplot(cnt)

In [ ]:
sorted_dif$express <- "NO"
sorted_dif$express [sorted_dif$log2FoldChange > 1  & sorted_dif$padj < 0.05] <- "UP"
sorted_dif$express [sorted_dif$log2FoldChange < -1 & sorted_dif$padj < 0.05] <- "DOWN"

diseased_vs_healthy <- sorted_dif %>%
  mutate(gene_type = case_when(log2FoldChange >= 1 & padj <= 0.05 ~ "up",
                               log2FoldChange <= -1 & padj <= 0.05 ~ "down",
                               TRUE ~ "ns")) 


ggplot(sorted_dif, aes(log2FoldChange, -log10(padj))) +
  geom_point() + 
  geom_hline(yintercept = -log10(0.05),
             linetype = "dashed") + 
  geom_vline(xintercept = c(log2(0.5), log2(2)))

cols <- c("up" = "#ffad73", "down" = "#26b3ff", "ns" = "grey") 
sizes <- c("up" = 2, "down" = 2, "ns" = 1) 
alphas <- c("up" = 1, "down" = 1, "ns" = 0.5)

diseased_vs_healthy %>%
  ggplot(aes(x = log2FoldChange,
             y = -log10(padj),
             fill = gene_type,    
             size = gene_type,
             alpha = gene_type)) + 
  geom_point(shape = 21, # Specify shape and colour as fixed local parameters    
             colour = "black") + 
  geom_hline(yintercept = -log10(0.05),
             linetype = "dashed") + 
  geom_vline(xintercept = c(log2(0.5), log2(2)),
             linetype = "dashed") +
  scale_fill_manual(values = cols) + # Modify point colour
  scale_size_manual(values = sizes) + # Modify point size
  scale_alpha_manual(values = alphas) + # Modify point transparency
  scale_x_continuous(breaks = c(seq(-10, 10, 2)),       
                     limits = c(-10, 10))  


## WGCNA